In [1]:
pip install alpha_vantage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
# from pprint import pprint
import requests
from datetime import datetime

# 1

In [4]:
class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        api_key = 'TCWNU2ZM9THSTMQ1'
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            self.data[script] = response.json()
        else:
            raise Exception("Failed to fetch data from Alpha Vantage")
    
    def convert_intraday_data(self, script):
        raw_data = self.data[script]["Time Series (5min)"]
        data = []
        for timestamp, values in raw_data.items():
            data.append([pd.Timestamp(timestamp), float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]), int(values["5. volume"])])
        self.data[script] = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    
    def __getitem__(self, script):
        return self.data[script]
    
    def __setitem__(self, script, df):
        self.data[script] = df
    
    def __contains__(self, script):
        return script in self.data

In [5]:
script_data = ScriptData()

In [6]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

In [7]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,94.3500,94.40,94.28,94.4000,13261
1,2023-02-13 19:55:00,94.3500,94.40,94.35,94.3500,6407
2,2023-02-13 19:50:00,94.2800,94.39,94.28,94.3500,2081
3,2023-02-13 19:45:00,94.4200,94.44,94.29,94.2900,2133
4,2023-02-13 19:40:00,94.4700,94.47,94.35,94.3500,917
...,...,...,...,...,...,...
95,2023-02-13 12:05:00,94.2900,94.31,94.22,94.2900,398928
96,2023-02-13 12:00:00,94.2701,94.31,94.24,94.3000,467125
97,2023-02-13 11:55:00,94.2500,94.30,94.12,94.2725,553289
98,2023-02-13 11:50:00,94.3064,94.36,94.23,94.2500,503563


In [8]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

In [9]:
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,153.660,153.7000,153.60,153.700,7841
1,2023-02-13 19:55:00,153.640,153.6700,153.64,153.660,4691
2,2023-02-13 19:50:00,153.650,153.6500,153.61,153.630,3566
3,2023-02-13 19:45:00,153.650,153.6700,153.65,153.670,2083
4,2023-02-13 19:40:00,153.650,153.6600,153.65,153.660,6946
...,...,...,...,...,...,...
95,2023-02-13 12:05:00,153.440,153.6700,153.41,153.570,477667
96,2023-02-13 12:00:00,153.550,153.5599,153.28,153.420,528881
97,2023-02-13 11:55:00,153.650,153.6900,153.51,153.550,598249
98,2023-02-13 11:50:00,153.510,153.6700,153.45,153.645,614682


In [10]:
'GOOGL' in script_data

True

In [11]:
'AAPL' in script_data

True

In [12]:
'NVDA' in script_data

False

# 2

In [50]:
def indicator1(df, timeperiod = 5):
    close_data = df["close"]
    indicator = close_data.rolling(window=timeperiod).mean()
    return pd.DataFrame({"timestamp": df["timestamp"], "indicator": indicator})

In [52]:
indicator1(script_data['GOOGL'], timeperiod = 5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,94.34800
...,...,...
95,2023-02-13 12:05:00,94.37956
96,2023-02-13 12:00:00,94.34656
97,2023-02-13 11:55:00,94.31106
98,2023-02-13 11:50:00,94.28206


In [53]:
indicator1(script_data['AAPL'], timeperiod = 5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,153.66400
...,...,...
95,2023-02-13 12:05:00,153.83488
96,2023-02-13 12:00:00,153.68798
97,2023-02-13 11:55:00,153.60598
98,2023-02-13 11:50:00,153.56898


# 3

In [136]:
import pandas as pd

class Strategy:
    def __init__(self, script):
        self.script = script
        self.script_data = ScriptData()
        if script not in self.script_data:
            self.script_data.fetch_intraday_data(script)
            self.script_data.convert_intraday_data(script)
        
    def get_script_data(self):
        self.df = self.script_data[self.script]
        # print(script_data)
        # print(self.df)
        # self.close_data = self.df['close']
    
    def get_signals(self):
        # print(self.close_data)
        indicator_data = indicator1(self.df['close'])
        print(self.df['close'])
        signals = pd.DataFrame({'timestamp': self.df['timestamp']})

        conditions = [indicator_data.shift() < self.df['close'].shift(), indicator_data.shift() > self.df['close'].shift()]
        choices = ['BUY', 'SELL']
        signals['signal'] = pd.Series(np.select(conditions, choices, default='NO_SIGNAL'))
        self.signals = signals



In [137]:
# class Strategy:
#     def __init__(self, script_data):
#         self.script_data = script_data
        
#     def indicator1(self, close_data):
#         indicator_data = close_data.rolling(window=3).mean()
#         return indicator_data
    
#     def get_signals(self, script):
#         df = self.script_data[script]
#         close_data = df['close']
#         indicator_data = self.indicator1(close_data)
#         signals = pd.DataFrame(index=df.index, columns=['signal'])
#         signals.loc[(indicator_data > close_data) & (indicator_data.shift(1) <= close_data.shift(1)), 'signal'] = 'BUY'
#         signals.loc[(indicator_data < close_data) & (indicator_data.shift(1) >= close_data.shift(1)), 'signal'] = 'SELL'
#         signals.loc[signals['signal'].isna(), 'signal'] = 'NO_SIGNAL'
#         return signals


In [138]:
strategy = Strategy('NVDA')

In [139]:
strategy.get_script_data()

In [140]:
strategy.get_signals()

KeyError: ignored

# 4

In [118]:
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
!pip install pyalgotrading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
import mplfinance as mpf
import pyalgotrading as pat

In [121]:
def get_indicator_addplot(df, indicator_data):
    indicator_addplot = mpf.make_addplot(indicator_data, panel=1, color='orange', ylabel='Indicator')
    indicator_addplot.panel = 1
    return indicator_addplot

In [122]:
def get_marketcolors():
    mc = mpf.make_marketcolors(up='green', down='red')
    return mc

In [123]:
def get_mpf_style():
    style = mpf.make_mpf_style(marketcolors=get_marketcolors())
    return style

In [124]:
def plot_candlestick_chart(df, indicator_data):
    mpf.plot(df, type='candle', addplot=get_indicator_addplot(df, indicator_data), style=get_mpf_style())

In [125]:
plot_candlestick_chart(df, indicator_data)

NameError: ignored